In [22]:
from pathlib import Path

import numpy as np
import pandas as pd

from lift.utils import read_csv
from lift.backend.scenario import SingleScenario, ScenarioResult

In [23]:
def run_casestudy(casestudy_name: str, casestudy_df: pd.DataFrame):
    results = {scn: SingleScenario.from_series(scn_series).simulate() for scn, scn_series in casestudy_df.iterrows()}

    def costs4plotting(result: ScenarioResult) -> np.ndarray:
        costs = np.zeros(2 * (result.period_eco + 1))
        costs[2::2] = result.opex_dis[:-1]
        costs[1::2] = result.capex_dis
        return np.cumsum(costs)

    df_time = pd.DataFrame(data={k: costs4plotting(v) for k, v in results.items()}) * 1e-6
    df_time["year"] = np.repeat(np.arange(len(df_time) // 2), 2)
    df_time = df_time[["year"] + [col for col in df_time.columns if col != "year"]]
    df_time.to_csv(f"{casestudy_name}_data_time.csv", index=False)

    df_kpis = pd.DataFrame(
        {
            k: {
                "opex": v.opex_dis.sum() * 1e-6,
                "capex": v.capex_dis.sum() * 1e-6,
                "gamma_sc": round(v.self_consumption, 2),
                "gamma_ss": round(v.self_sufficiency, 2),
                "gamma_ch": round(v.home_charging_fraction, 2),
                "co2": v.totem.sum() * 1e-6,
            }
            for k, v in results.items()
        }
    ).T
    df_kpis.index.name = "scenario"
    df_kpis.to_csv(f"{casestudy_name}_data_total.csv")

In [24]:
df = read_csv(Path("case_a.csv"), transpose=True)
run_casestudy("A", df)
df = read_csv(Path("case_b.csv"), transpose=True)
run_casestudy("B", df)

Scenario simulation time: 0.0400 s
Scenario simulation time: 0.6256 s
Scenario simulation time: 0.6220 s
Scenario simulation time: 0.0312 s
Scenario simulation time: 0.8790 s
Scenario simulation time: 0.8900 s
